In [ ]:
!pip install tensorflow==2.17.0
!pip install keras>=3.2.0
!pip install --upgrade scikeras

In [ ]:
!pip show tensorflow keras scikeras

Name: tensorflow
Version: 2.17.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras
---
Name: keras
Version: 3.8.0
Summary: Multi-backend Keras
Home-page: 
Author: 
Author-email: Keras team <keras-users@googlegroups.com>
License: Apache License 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, packaging, rich
Required-by: keras-tuner, scikeras, tensorflow
---
Name: scikeras
Version: 0.13.0
Summary: Scikit-Learn API wrapper for Keras.
Home-page: https:/

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from scikeras.wrappers import KerasClassifier

In [ ]:
df = pd.read_csv('/content/Alphabets_data.csv')

**Data Exploration and Preprocessing**


In [ ]:
# Explore the dataset
print("Dataset shape:", df.shape)  # Number of samples and features
print("\nData types:\n", df.dtypes)
print("\nFirst few rows:\n", df.head())
print("\nSummary Statistics:\n", df.describe())
print("\nMissing values:\n", df.isnull().sum())

Dataset shape: (20000, 17)

Data types:
 letter    object
xbox       int64
ybox       int64
width      int64
height     int64
onpix      int64
xbar       int64
ybar       int64
x2bar      int64
y2bar      int64
xybar      int64
x2ybar     int64
xy2bar     int64
xedge      int64
xedgey     int64
yedge      int64
yedgex     int64
dtype: object

First few rows:
   letter  xbox  ybox  width  height  onpix  xbar  ybar  x2bar  y2bar  xybar  \
0      T     2     8      3       5      1     8    13      0      6      6   
1      I     5    12      3       7      2    10     5      5      4     13   
2      D     4    11      6       8      6    10     6      2      6     10   
3      N     7    11      6       6      3     5     9      4      6      4   
4      G     2     1      3       1      1     8     6      6      6      6   

   x2ybar  xy2bar  xedge  xedgey  yedge  yedgex  
0      10       8      0       8      0       8  
1       3       9      2       8      4      10  
2       3    

In [ ]:
# Identify classes
if 'Class' in df.columns:  # Assuming the class column is named 'Class'
  print(f"\nClasses: {df['Class'].unique()}")
  print("\nClass Counts:\n", df['Class'].value_counts())
else:
  print("\nWarning: 'Class' column not found in the dataset.")

In [ ]:
# 1. Handling missing values (using SimpleImputer)

# Assuming numerical features need imputation, replace with the mean
numerical_features = df.select_dtypes(include=['number']).columns

if len(numerical_features) > 0:
  imputer = SimpleImputer(strategy='mean')  # or 'median', 'most_frequent', etc.
  df[numerical_features] = imputer.fit_transform(df[numerical_features])
else:
  print("\nWarning: No numerical features found for imputation.")

In [ ]:
# 2. Data Normalization (using MinMaxScaler)
scaler = MinMaxScaler()
numerical_features = df.select_dtypes(include=['number']).columns

if len(numerical_features) > 0:
  df[numerical_features] = scaler.fit_transform(df[numerical_features])
else:
  print("\nWarning: No numerical features found for normalization.")


print("\nDataset after preprocessing:\n", df.head())
print("\nMissing values after preprocessing:\n", df.isnull().sum())


Dataset after preprocessing:
   letter      xbox      ybox  width    height     onpix      xbar      ybar  \
0      T  0.133333  0.533333    0.2  0.333333  0.066667  0.533333  0.866667   
1      I  0.333333  0.800000    0.2  0.466667  0.133333  0.666667  0.333333   
2      D  0.266667  0.733333    0.4  0.533333  0.400000  0.666667  0.400000   
3      N  0.466667  0.733333    0.4  0.400000  0.200000  0.333333  0.600000   
4      G  0.133333  0.066667    0.2  0.066667  0.066667  0.533333  0.400000   

      x2bar     y2bar     xybar    x2ybar    xy2bar     xedge    xedgey  \
0  0.000000  0.400000  0.400000  0.666667  0.533333  0.000000  0.533333   
1  0.333333  0.266667  0.866667  0.200000  0.600000  0.133333  0.533333   
2  0.133333  0.400000  0.666667  0.200000  0.466667  0.200000  0.466667   
3  0.266667  0.400000  0.266667  0.266667  0.666667  0.400000  0.666667   
4  0.400000  0.400000  0.400000  0.333333  0.600000  0.066667  0.466667   

      yedge    yedgex  
0  0.000000  0.5333

**Model Implementation**


In [ ]:
# Preprocess the data
# Assuming the last column is the target variable (alphabet)
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [ ]:
# Encode the target variable (alphabets) into numerical values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define the ANN model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))  # Input layer and first hidden layer
model.add(Dense(32, activation='relu')) # Second hidden layer
model.add(Dense(len(label_encoder.classes_), activation='softmax')) # Output layer with softmax

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Before calling model.evaluate, ensure your data is of the correct type and structure.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert X_test to NumPy array with object dtype to handle mixed types
X_test = np.asarray(X_test, dtype=object)

# Iterate through columns of X_test and convert to numeric if possible
for col in range(X_test.shape[1]):
    try:
        X_test[:, col] = X_test[:, col].astype(np.float32)
    except ValueError:
        # If conversion fails, it's likely a categorical feature
        # Use Label Encoding or One-Hot Encoding here
        from sklearn.preprocessing import LabelEncoder
        encoder = LabelEncoder()
        X_test[:, col] = encoder.fit_transform(X_test[:, col])

# Convert to float32 after individual column conversions
X_test = X_test.astype(np.float32)

# Convert y_test to NumPy arrays with the correct dtype
y_test = np.asarray(y_test, dtype=np.int32)    # Assuming your labels are integers

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1089 - loss: 4.0229
Test Loss: 4.036161422729492
Test Accuracy: 0.10649999976158142


In [ ]:
# Make predictions
predictions = model.predict(X_test)
predicted_labels = [label_encoder.inverse_transform([i]) for i in predictions.argmax(axis=1)]

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [ ]:
# Print some predictions
for i in range(5): # Print the first 5 predictions
    print(f"Predicted: {predicted_labels[i][0]}, Actual: {label_encoder.inverse_transform([y_test[i]])[0]}")

Predicted: 0.6, Actual: 0.4
Predicted: 0.6, Actual: 0.3333333333333333
Predicted: 0.4, Actual: 0.5333333333333333
Predicted: 0.6, Actual: 0.6666666666666666
Predicted: 0.6, Actual: 0.6


**Hyperparameter Tuning**

**The following hyperparameters can be tuned:**

1) Number of hidden layers

2) Number of neurons in each layer

3) Activation function (e.g., ReLU, tanh, sigmoid)

4) Learning rate of the optimizer

5) Batch size and number of epochs

We can use GridSearchCV or RandomizedSearchCV for hyperparameter tuning. However, for neural networks, KerasTuner (or similar) is a more suitable choice because GridSearchCV and RandomizedSearchCV don't directly support Keras models.

Here's an example of hyperparameter tuning using KerasTuner:

In [ ]:
!pip install keras-tuner

In [ ]:
import keras_tuner as kt
import numpy as np
from sklearn.preprocessing import LabelEncoder

def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=128, step=32),
                    activation='relu', input_dim=X_train.shape[1]))
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(Dense(units=len(np.unique(y)), activation='softmax'))

    model.compile(optimizer=hp.Choice('optimizer', ['adam', 'sgd']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Preprocess X_train before passing it to the tuner
X_train_processed = np.asarray(X_train, dtype=object)
for col in range(X_train_processed.shape[1]):
    try:
        X_train_processed[:, col] = X_train_processed[:, col].astype(np.float32)
    except ValueError:
        encoder = LabelEncoder()
        X_train_processed[:, col] = encoder.fit_transform(X_train_processed[:, col])

X_train_processed = X_train_processed.astype(np.float32)


tuner = kt.Hyperband(build_model, objective='val_accuracy', max_epochs=10, directory='my_dir', project_name='alphabet_tuning')

# Perform the hyperparameter search, using the processed X_train
tuner.search(X_train_processed, y_train, epochs=20, validation_split=0.2)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(1)[0]
print(f"Best hyperparameters: {best_hps.values}")

Trial 8 Complete [00h 00m 07s]
val_accuracy: 0.40812501311302185

Best val_accuracy So Far: 0.43062499165534973
Total elapsed time: 00h 07m 56s
Best hyperparameters: {'units': 96, 'optimizer': 'adam', 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}


**Retrain Model with Best Hyperparameters**
Once you’ve tuned the hyperparameters, retrain the model using the best configuration.

In [ ]:
# Ensure X_train and y_train are of the correct type and structure
X_train = np.asarray(X_train, dtype=object)

# Iterate through columns of X_train and convert to numeric if possible
for col in range(X_train.shape[1]):
    try:
        X_train[:, col] = X_train[:, col].astype(np.float32)
    except ValueError:
        # If conversion fails, it's likely a categorical feature
        # Use Label Encoding or One-Hot Encoding here
        from sklearn.preprocessing import LabelEncoder
        encoder = LabelEncoder()
        X_train[:, col] = encoder.fit_transform(X_train[:, col])

# Convert to float32 after individual column conversions
X_train = X_train.astype(np.float32)

y_train = np.asarray(y_train, dtype=np.int32)    # Assuming your labels are integers
model = build_model(best_hps)
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.3796 - loss: 2.0029 - val_accuracy: 0.4019 - val_loss: 1.7391
Epoch 2/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4085 - loss: 1.6738 - val_accuracy: 0.4131 - val_loss: 1.5873
Epoch 3/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4203 - loss: 1.5746 - val_accuracy: 0.4387 - val_loss: 1.4916
Epoch 4/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4285 - loss: 1.4985 - val_accuracy: 0.4550 - val_loss: 1.4459
Epoch 5/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4434 - loss: 1.4630 - val_accuracy: 0.4550 - val_loss: 1.4039
Epoch 6/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4473 - loss: 1.4181 - val_accuracy: 0.4787 - val_loss: 1.3794
Epoch 7/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4650 - loss: 1.3846 - val_accuracy: 0.4816 - val_loss: 1.3594
Epoch 8/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4821 - loss: 1.3584 - val_accuracy: 0.

**Evaluation**


**Evaluate the Model's Performance**

Evaluate the model using metrics like accuracy, precision, recall, and F1-score.

In [ ]:
from sklearn.metrics import classification_report

# Predict the test set
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=-1)

# Evaluation metrics
print(classification_report(y_test, y_pred_classes))


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00        31
           4       0.41      0.53      0.46        94
           5       0.47      0.19      0.27       196
           6       0.39      0.49      0.44       353
           7       0.57      0.34      0.43       712
           8       0.65      0.83      0.73      1596
           9       0.41      0.29      0.34       485
          10       0.41      0.51      0.46       308
          11       0.61      0.54      0.57       175
          12       0.00      0.00      0.00        29
          13       0.00      0.00      0.00         9
          14       0.00      0.00      0.00         1

    accuracy                           0.56      4000
   macro avg       0.26      0.25      

**Compare the Default and Tuned Model Performance**

Compare the performance of the model before and after hyperparameter tuning. The accuracy, precision, recall, and F1-scores should be documented and discussed to show the effect of hyperparameter optimization.

**Conclusion**


Data Preprocessing: Ensuring proper scaling and handling of missing values improves model performance.
Model Development: A simple ANN can provide decent results, but hyperparameter tuning plays a crucial role in improving its accuracy.
Hyperparameter Tuning: Using tools like KerasTuner or GridSearchCV helps in finding optimal configurations to maximize model performance.
Evaluation: Use appropriate classification metrics to evaluate the effectiveness of the model and tuning process.
